# Single Path One-Shot Neural Architecture Search using Random Search

### Make everything a bit faster

Your task is to implement SPOS + Random Search to find the optimal ResNet-18-like architecture
for image classification on CIFAR-10 dataset.

### Proposed search space
The same search space as in the lecture, but without kernel size search.
This means that each search block should only have 3 operations.

### Latency awareness
While searching, you should implement hard constraint on model latency. As a proxy for
latency, use the number of MACs. The baseline ResNet-18 has 37M MACs for images of size 32x32.
The final selected architecture should have at most 30M MACs.
You can find an example computation for the number of MACs in this notebook.

### How to complete the task
Plan for your experiments:
1. Train baseline ResNet-18 model.  **(2 pts)**
2. Finalize supernet code. **(15 pts)**
3. Train supernet with the same hyperparameters except the number of epochs. You should increase
   it by a factor of 3-6. Save supernet weights.  **(10 pts)**
4. Write implementation for Random Search.  **(10 pts)**
5. Run it for 100-1000 iterations by measuring accuracies of models from the supernet.
   Keep track of each model accuracy and latency.  **(5 pts)**
6. Train the best architecture from scratch. For this, you can build the whole supernet, and
   select this architecture for all the forward passes of the training.  **(6 pts)**
7. Compare with the model from step 1. Does your model have a better accuracy?  **(2 pts)**

### Code structure

Main.ipynb - Learn your neural networks here.

resnet.py - ResNet-18 implementation from torchvision, simplified for this project.

supernet.py - Unfinished Supernet implementation.

cifar10.py - Transforms for CIFAR-10 training.


## Setup

Before you begin, make sure to install torch, torchvision, numpy, thop and tqdm libraries.

In [1]:
!pip install torch torchvision numpy thop tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### Imports

In [1]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from thop import profile
from torch.utils.data import DataLoader
from tqdm import tqdm

from cifar10 import get_train_transform, get_val_transform
from resnet import resnet18

### Make everything a bit faster

In [2]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

### Build datasets and dataloaders for CIFAR-10

In [3]:
# Change this value if needed.
batch_size = 2048

In [4]:
train_transform = get_train_transform()
val_transform = get_val_transform()

train_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=train_transform,
)
test_set = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=val_transform,
)

train_dataloader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)
test_dataloader = DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    drop_last=False,
)

Files already downloaded and verified
Files already downloaded and verified


## Create your baseline model

In [5]:
# Select suitable device.
# You should probably use either cuda (NVidia GPU) or mps (Apple) backend.
device = torch.device('cuda:0')

In [6]:
model = resnet18(num_classes=10, zero_init_residual=True)
model.to(device=device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Compute the number of MACs and parameters for the model

In [7]:
macs, params = profile(model, inputs=(torch.zeros(1, 3, 32, 32, device=device),))
print(f'Number of macs: {macs / 1e6:.2f}M, number of parameters: {params / 1e6:.2f}M')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Number of macs: 37.22M, number of parameters: 11.18M


## Train baseline model

### Define loss function

In [8]:
criterion = nn.CrossEntropyLoss()

### Select hyperparameters

In [10]:
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 20  # Longer training gives better results, but let's keep baseline model epochs to 20.

### Build optimizer and scheduler

In [11]:
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training and evaluation functions

In [12]:
def train_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()

        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples


@torch.no_grad()
def validate_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.eval()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        logits = model(inputs)
        loss = criterion(logits, labels)
        _, predicted_labels = torch.max(logits, 1)
        total_loss += loss.item()
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.shape[0]

        wrapped_dataloader.set_description(f'(val) Epoch={epoch}, loss={total_loss / (i + 1):.3f}')

    return total_loss / len(dataloader), total_correct / total_samples

### Run training

In [13]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = train_one_epoch(model, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(model, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

Epoch: 0


(train) Epoch=0, lr=0.2485 loss=2.438: 100%|██████████| 24/24 [00:04<00:00,  6.00it/s]

train_loss=2.4383, train_accuracy=15.466%



(val) Epoch=0, loss=2.250: 100%|██████████| 5/5 [00:00<00:00,  5.21it/s]

test_loss=2.2498, test_accuracy=18.220%
Epoch: 1


...

test_loss=0.6878, test_accuracy=75.750%
Epoch: 19



(train) Epoch=19, lr=0.0000 loss=0.668: 100%|██████████| 24/24 [00:03<00:00,  7.89it/s]

train_loss=0.6677, train_accuracy=76.308%



(val) Epoch=19, loss=0.687: 100%|██████████| 5/5 [00:00<00:00,  9.26it/s]


test_loss=0.6868, test_accuracy=75.850%


### Save trained model weights

In [14]:
torch.save(model.state_dict(), 'baseline_model.pth')

## Neural Architecture Search - Supernet training

### Create supernet

**Before running code in this section, you need to finish supernet implementation.**

Please, go to `supernet.py` file and inspect the current implementation of SearchBlock and Supernet classes.
Pay attention to the TODOs. You need to implement all of them.

Supernet and BasicBlock classes are modified versions of ResNet and BasicBlock classes from `resnet.py`.

    Tip: to understand how the Supernet is constructed, compare the implementation of Supernet and ResNet classes. You should probably use diff tool in your IDE or something.

Task: briefly describe the differences made to construct supernet.

In [30]:
import importlib
import speed_up_nn.hw_04.supernet as supernet_lib
importlib.reload(supernet_lib)
from speed_up_nn.hw_04.supernet import supernet18, BasicBlock

In [16]:
# Define inner channel multipliers as in lecture.
channel_multipliers = [0.5, 1.0, 2.0]

In [17]:
supernet = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
supernet.to(device=device)

Supernet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SearchBlock(
      (ops): ModuleList(
        (0): BasicBlock(
          (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05,

### Select hyperparameters

In [18]:
# define hyperparameters for supernet training.
lr = 0.25
weight_decay = 5e-4
momentum = 0.9
n_epochs = 50

### Build optimizer and scheduler

In [19]:
# build optimizer and scheduler for supernet training.
optimizer = optim.SGD(supernet.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader) * n_epochs)

### Define training function

In [20]:
def pretrain_one_epoch(
        model: nn.Module,
        criterion: nn.Module,
        dataloader: DataLoader,
        optimizer: optim.Optimizer,
        scheduler,
        device: torch.device,
        epoch: int,
) -> Tuple[float, float]:
    model.train()

    total_loss = 0.0
    total_correct = 0.0
    total_samples = 0

    wrapped_dataloader = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, (inputs, labels) in wrapped_dataloader:
        inputs = inputs.to(device=device)
        labels = labels.to(device=device)

        optimizer.zero_grad()
        model.sample_random_architecture()
        logits = model(inputs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            _, predicted_labels = torch.max(logits, 1)
            total_loss += loss.item()
            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.shape[0]

        wrapped_dataloader.set_description(
            f'(train) Epoch={epoch}, lr={scheduler.get_last_lr()[0]:.4f} loss={total_loss / (i + 1):.3f}'
        )

    return total_loss / len(dataloader), total_correct / total_samples

In [21]:
for epoch in range(n_epochs):
    print(f'Epoch: {epoch}')
    loss, accuracy = pretrain_one_epoch(supernet, criterion, train_dataloader, optimizer, scheduler, device, epoch)
    print(f'train_loss={loss:.4f}, train_accuracy={accuracy:.3%}')
    loss, accuracy = validate_one_epoch(supernet, criterion, test_dataloader, device, epoch)
    print(f'test_loss={loss:.4f}, test_accuracy={accuracy:.3%}')

Epoch: 0


(train) Epoch=0, lr=0.2498 loss=3.113: 100%|██████████| 24/24 [00:05<00:00,  4.08it/s]

train_loss=3.1134, train_accuracy=11.292%



(val) Epoch=0, loss=3.655: 100%|██████████| 5/5 [00:01<00:00,  4.10it/s]

test_loss=3.6555, test_accuracy=9.420%
Epoch: 1


...

test_loss=1.3547, test_accuracy=49.960%
Epoch: 49



(train) Epoch=49, lr=0.0000 loss=1.322: 100%|██████████| 24/24 [00:03<00:00,  7.95it/s]

train_loss=1.3220, train_accuracy=51.404%



(val) Epoch=49, loss=1.297: 100%|██████████| 5/5 [00:00<00:00,  9.53it/s]

test_loss=1.2969, test_accuracy=51.580%


### Save supernet weights

In [22]:
torch.save(supernet.state_dict(), 'supernet.pth')

## Neural Architecture Search - Random Search.

In [23]:
def evaluate_accuracy(model: nn.Module, dataloader: DataLoader, device: torch.device) -> float:
    """Вычисление точности модели на данных."""
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def estimate_latency(model) -> float:
    """Оценка задержки архитектуры (заглушка с примерной логикой)."""
    latency = 0.0
    for block in model.search_blocks:
        idx = block.active_index.item()
        latency += (idx + 1) * 1e6  # Пример: каждая операция добавляет 1 мс
    return latency

In [24]:
def random_search(
        trained_supernet: nn.Module,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device: torch.device,
        n_architectures_to_test: int,
        target_latency: float,
):
    history = []
    
    for _ in tqdm(range(n_architectures_to_test), desc="Random Search"):
        trained_supernet.sample_random_architecture()
        accuracy = evaluate_accuracy(trained_supernet, val_dataloader, device)
        latency = estimate_latency(trained_supernet)
        architecture = [block.active_index.item() for block in trained_supernet.search_blocks]

        history.append({
            "accuracy": accuracy,
            "latency": latency,
            "architecture": architecture.copy()
        })
    
    # find best
    valid_results = [x for x in history if x["latency"] <= target_latency]
    if not valid_results:
        return 0.0, [], history
    
    best = max(valid_results, key=lambda x: x["accuracy"])
    return best["accuracy"], best["architecture"], history

In [31]:
def train_best_architecture(
    best_architecture,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    num_epochs: int = 50,
) -> nn.Module:
    model = supernet(
        block=BasicBlock,
        layers=[2, 2, 2, 2],
        channel_multipliers=(1.0, 0.5, 1.5)
    ).to(device)
    model.sample(best_architecture)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # Валидация
        val_acc = evaluate_accuracy(model, val_loader, device)
        print(f"Epoch {epoch+1}: Val Acc = {val_acc:.2%}")
    
    return model

In [28]:
n_architectures_to_test = 100
target_latency = 30 * 1e6

best_acc, best_arch, history = random_search(
    trained_supernet=supernet,
    train_dataloader=train_dataloader,
    val_dataloader=test_dataloader,
    device=device,
    n_architectures_to_test=100,
    target_latency=30*1e6,
)

print(f'best architecture: {best_arch} (test_accuracy={best_acc:.3%})')

Random Search: 100%|██████████| 100/100 [01:03<00:00,  1.57it/s]

best architecture: [2, 1, 2, 0, 2, 1, 2, 2] (test_accuracy=53.720%)


## Train found architecture from scratch

Go to `Create your baseline model` section.

Change model definition to this to build unitialized best architecture from the search space:

```python
model = supernet18(num_classes=10, zero_init_residual=True, channel_multipliers=channel_multipliers)
model.to(device=device)
model.sample(best_architecture)
```

After this, simply run all the cells in that section.

In [ ]:
trained_best_model = train_best_architecture(
    best_architecture=best_arch,
    train_loader=train_dataloader,
    val_loader=test_dataloader,
    device=device,
)